<a href="https://www.kaggle.com/code/douglashsm/llm-crutch?scriptVersionId=263271673" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# LLM Crutch 🤖

> ### Tired of asking your data questions and getting... guesses? So were we.

**A Conversational Business Intelligence Agent for the BigQuery AI Hackathon that doesn't just answer questions—it finds the truth.**

---

### 🚀 See it in Action!

Before you dive in, see the magic happen first-hand.

* **[==(https://youtu.be/3RJd8icgh6g)** (👈 Link your 2-3 minute video here)
* **https://muleta-llm.streamlit.app/** (👈 The ngrok link generated by your notebook)

### 🎯 The Billion-Dollar Problem: AI's Trust Deficit

Generative AI is a revolution. But in the world of data, it has a dirty little secret: **it lies**. Not maliciously, but it "hallucinates," providing confident-sounding answers that are subtly wrong or completely fabricated. For a business relying on data for critical decisions, "mostly right" is 100% wrong.

How can we harness the incredible conversational power of LLMs without inheriting their unreliability?

### 💡 Our Solution: The "LLM Crutch"

We didn't try to build a "smarter" AI. We built a *wiser* one.

**LLM Crutch** is an architectural pattern where the LLM is not the oracle, but a brilliant **interpreter**. It leans on a "crutch"—the unshakable, high-performance foundation of **Google BigQuery**—to ensure every single answer is grounded in factual, verifiable data.

Our agent, **QueryMaster**, embodies this philosophy. It's a conversational partner that listens to your business question, reasons about your intent, translates it into sophisticated SQL, delegates the heavy lifting to BigQuery, and presents the results beautifully. It's a `conversation-to-insight` pipeline you can actually trust.

### ✨ Key Features

* **🧠 True Conversational Logic:** The agent asks clarifying questions for vague prompts, just like a real data analyst.
* **🚀 Complex SQL on Demand:** Generates sophisticated queries with `JOIN`s, complex calculations, and aggregations from natural language.
* **🎨 AI-Advised Visualizations:** The LLM suggests the display format for the data (`currency`, `percentage`), creating a smarter and simpler front-end.
* **🔍 Full Transparency:** The exact SQL query used is always displayed, building trust in the result.
* **⚡ Interactive & Real-Time:** Built with Streamlit, the interface is fast and allows for a seamless analytical dialogue.

### 🛠️ Architecture & Tech Stack

Our "self-contained agent" model is designed for performance and simplicity, running entirely within a notebook environment.

**Flow:**
> User → `Streamlit UI` → `Gemini API` → (Generates SQL) → `BigQuery API` → (Returns Data) → `Streamlit UI`

**Tech Stack:**
* **Front-End & Orchestration:** `Streamlit`
* **AI Reasoning Engine:** `Google Gemini 1.5 Flash`
* **Data Warehouse & Processing:** `Google BigQuery`
* **Data Handling & Charting:** `Pandas`, `Plotly Express`

### 🔮 Future Vision

The "LLM Crutch" pattern is endlessly extensible. The next steps are clear:
* **Forecasting Integration:** Leverage `BigQuery ML` to answer questions about the future.
* **Multimodal Crutches:** Allow the agent to ingest unstructured data from the web (e.g., customer sentiment) and correlate it with hard sales data from BigQuery.

---
*A project developed for the BigQuery AI Hackathon. We believe the future of data is about building smarter, more reliable systems that bridge the gap between human questions and verifiable truth.*
=======



In [1]:
!pip install -q streamlit google-generativeai google-cloud-bigquery pandas db-dtypes plotly pyngrok tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.5 MB/s eta 0:00:00


In [2]:
%%writefile app.py

# ===============================================================
# app.py - FINAL SUBMISSION VERSION (CORRECTED INDENTATION)
# ===============================================================

import streamlit as st
import google.generativeai as genai
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import json
import pandas as pd
import plotly.express as px
from kaggle_secrets import UserSecretsClient

# --- PAGE CONFIGURATION ---
st.set_page_config(
    page_title="QueryMaster AI for BigQuery",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- CSS TO HIDE STREAMLIT'S DEFAULT MENU AND FOOTER ---
hide_st_style = """
            <style>
            #MainMenu {visibility: hidden;}
            footer {visibility: hidden;}
            </style>
            """
st.markdown(hide_st_style, unsafe_allow_html=True)


# --- 1. SETUP AND AUTHENTICATION ---
try:
    user_secrets = UserSecretsClient()
    google_api_key = user_secrets.get_secret("GOOGLE_API_KEY")
    gcp_key_content = user_secrets.get_secret("GCP_KEY")

    genai.configure(api_key=google_api_key)
    
    gcp_key_json = json.loads(gcp_key_content)
    credentials = service_account.Credentials.from_service_account_info(gcp_key_json)
    client_bq = bigquery.Client(credentials=credentials, project=credentials.project_id)
    
    st.session_state.auth_success = True
except Exception as e:
    st.error(f"Authentication Error. Please check your Kaggle Secrets. Details: {e}")
    st.session_state.auth_success = False
    st.stop()
    
# --- SYSTEM INSTRUCTION FOR THE LLM ---
SYSTEM_INSTRUCTION = """
You are 'QueryMaster', an AI Data Analyst specializing in the Google BigQuery 'TheLook' e-commerce dataset.
Your mission is to transform business questions into valid BigQuery Standard SQL queries.

You will use the following main schema:
CREATE TABLE `bigquery-public-data.thelook_ecommerce.order_items` (
  order_id STRING, user_id STRING, product_id STRING, sale_price NUMERIC, created_at TIMESTAMP
);
CREATE TABLE `bigquery-public-data.thelook_ecommerce.products` (
  id STRING, cost NUMERIC, category STRING, name STRING, brand STRING, department STRING
);
CREATE TABLE `bigquery-public-data.thelook_ecommerce.users` (
  id STRING, email STRING, first_name STRING, last_name STRING
);

Your response MUST be a valid JSON object.
The JSON must have the keys "action" and "content".
- For vague questions, use: {"action": "CLARIFY", "content": "Your clarification question here."}
- To generate SQL, use: {"action": "EXECUTE", "content": "Your SQL query here.", "display_format": "..."}

Possible values for "display_format": "currency_usd", "percentage", "number".

IMPORTANT: To group data by month and year from a TIMESTAMP column like 'created_at', use the function FORMAT_TIMESTAMP('%Y-%m', created_at). NEVER use the strftime function.
"""

# Continue only if authentication was successful
if 'auth_success' in st.session_state and st.session_state.auth_success:
    model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=SYSTEM_INSTRUCTION)
    
    # --- 2. BACKEND LOGIC (HELPER FUNCTIONS) ---

    def clean_json_from_string(text):
        """Extracts a JSON string from within a Markdown code block."""
        start_index = text.find('{')
        end_index = text.rfind('}')
        if start_index != -1 and end_index != -1:
            return text[start_index:end_index+1]
        return text

    # --- THIS FUNCTION IS NOW CORRECTED ---
    @st.cache_data
    def get_assistant_response(user_prompt, history_tuple):
        """
        This function encapsulates the backend logic. It calls the LLM and, if required, BigQuery.
        """
        history = [json.loads(item) for item in history_tuple]
        try:
            chat_session = model.start_chat(history=history)
            response = chat_session.send_message(user_prompt)
            cleaned_text = clean_json_from_string(response.text)
            
            response_json = json.loads(cleaned_text)
            action = response_json.get("action")

            if action != "EXECUTE":
                return response_json # If the action is CLARIFY, return immediately.

            # If the action IS EXECUTE, the code continues below.
            sql_query = response_json.get("content")
            display_format = response_json.get("display_format", "number")
            
            query_job = client_bq.query(sql_query)
            df_results = query_job.to_dataframe()
            
            return {
                "action": "DATA", 
                "content": df_results.to_dict('records'),
                "query_used": sql_query, 
                "display_format": display_format 
            }

        except json.JSONDecodeError:
            return {"action": "ERROR", "content": f"The model responded in an unexpected format: '{response.text}'"}
        except Exception as e:
            return {"action": "ERROR", "content": f"An error occurred: {e}"}

    def process_and_display_prompt(prompt):
        """Processes a prompt from the chat input or a button and displays the results in the UI."""
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Analyzing data..."):
                history_for_cache = tuple(json.dumps(item) for item in st.session_state.history_for_api)
                response_data = get_assistant_response(prompt, history_for_cache)
                action = response_data.get("action")
                if action == "CLARIFY":
                    message_content = response_data["content"]
                    st.markdown(message_content)
                    st.session_state.messages.append({"role": "assistant", "content": message_content})
                elif action == "DATA":
                    data_content = response_data["content"]
                    display_format = response_data.get("display_format", "number")
                    if not data_content:
                        st.warning("The query returned no results.")
                    else:
                        df = pd.DataFrame(data_content)
                        if df.shape[0] == 1 and df.shape[1] == 1:
                            st.markdown("#### Key Metric")
                            kpi_value = df.iloc[0, 0]
                            kpi_label = df.columns[0].replace("_", " ").title()
                            if display_format == "percentage": formatted_value = f"{kpi_value:,.2f}%"
                            elif display_format == "currency_usd": formatted_value = f"${kpi_value:,.2f}"
                            else: formatted_value = f"{kpi_value:,}"
                            st.metric(label=kpi_label, value=formatted_value)
                        else:
                            col1, col2 = st.columns([1, 1.2])
                            with col1:
                                st.markdown("#### Detailed Data")
                                st.dataframe(df)
                            with col2:
                                st.markdown("#### Chart")
                                try:
                                    x_axis, y_axis = df.columns[0], df.columns[1]
                                    fig = px.bar(df, x=x_axis, y=y_axis, title=f'{y_axis.replace("_", " ").title()} by {x_axis.replace("_", " ").title()}', template="seaborn")
                                    st.plotly_chart(fig, use_container_width=True)
                                except Exception:
                                    st.warning("Could not generate a chart for this data.")
                    with st.expander("View the generated SQL query"):
                        st.code(response_data["query_used"], language="sql")
                    message_content = "[Displaying data and charts]"
                    st.session_state.messages.append({"role": "assistant", "content": message_content})
                else:
                    message_content = f"An error occurred: {response_data.get('content')}"
                    st.error(message_content)
                    st.session_state.messages.append({"role": "assistant", "content": message_content})
        st.session_state.history_for_api.append({"role": "user", "parts": [prompt]})
        st.session_state.history_for_api.append({"role": "model", "parts": [json.dumps(response_data)]})

    # --- 3. STREAMLIT UI ---
    st.title("LLM Crutch 🤖: Your Data Analysis Assistant")
    st.caption("A project for the Kaggle BigQuery AI Hackathon by Douglas Menezes")
    st.sidebar.title("Analysis Suggestions 💡")
    st.sidebar.markdown("Click a button to ask a sample question!")
    if st.sidebar.button("📊 Monthly Profit (Chart)"): process_and_display_prompt("Show me the monthly profit evolution for the year 2023.")
    if st.sidebar.button("🏆 Top 5 Profitable Brands (Table)"): process_and_display_prompt("What are the 5 most profitable brands?")
    if st.sidebar.button("💰 Annual Growth (KPI %)") : process_and_display_prompt("What was the percentage revenue growth between 2022 and 2023?")
    if st.sidebar.button("🤯 Top Spenders Analysis (Complex JOIN)"): process_and_display_prompt("List the top 3 users (with their emails) who spent the most on 'Jeans' products.")
    st.sidebar.markdown("---")
    st.sidebar.title("Controls")
    if st.sidebar.button("🧹 Clear Conversation"):
        st.session_state.messages, st.session_state.history_for_api = [], []
        st.rerun()
    st.sidebar.markdown("---")
    st.sidebar.info("**About:** 'LLM Crutch' is a conversational BI tool using Google's Gemini AI to translate natural language into SQL queries, executed on BigQuery.")
    if "messages" not in st.session_state:
        st.session_state.messages, st.session_state.history_for_api = [], []
    for message in st.session_state.messages:
        with st.chat_message(message["role"]): st.markdown(message["content"])
    if prompt := st.chat_input("Ask your own question about the data..."):
        process_and_display_prompt(prompt)

Writing app.py


In [3]:
# ===============================================================
# CÉLULA LANÇADORA FINAL )
# ===============================================================

from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient
import subprocess
import threading
import time

# --- 1. CONFIGURAR O NGROK ---
try:
    user_secrets = UserSecretsClient()
    ngrok_token = user_secrets.get_secret("NGROK_AUTHTOKEN")
    ngrok.set_auth_token(ngrok_token)
    print("✅ Authtoken do ngrok configurado!")
except Exception as e:
    print(f"❌ Erro ao configurar o ngrok. Verifique seus Secrets. Detalhes: {e}")

# --- 2. FUNÇÃO PARA INICIAR O STREAMLIT EM OUTRA THREAD ---
def run_streamlit():
    # Usamos subprocess para chamar o comando do streamlit
    command = ["streamlit", "run", "app.py", "--server.port=8501", "--server.headless=true"]
    process = subprocess.Popen(command)
    process.wait()

# --- 3. INICIAR TUDO ---
# Criamos e iniciamos a "thread" para o Streamlit.
# Isso faz com que o Streamlit rode em paralelo, sem travar o notebook.
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.start()
print("✅ Servidor Streamlit iniciado em segundo plano!")

# Damos um pequeno tempo para o servidor Streamlit começar a esquentar
time.sleep(5)

# Agora que o Streamlit está rodando, iniciamos o túnel do ngrok
try:
    public_url = ngrok.connect(8501)
    print("---")
    print("🚀 Sua aplicação está no ar!")
    print(f"Clique neste link para acessar: {public_url}")
    print("---")
except Exception as e:
    print(f"❌ Erro ao criar o túnel do ngrok. Detalhes: {e}")

✅ Authtoken do ngrok configurado!
✅ Servidor Streamlit iniciado em segundo plano!



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://35.227.15.19:8501

❌ Erro ao criar o túnel do ngrok. Detalhes: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.


ERROR:  authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.
ERROR:  You can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.
ERROR:  Read more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config
ERROR:  You can view your current agent sessions in the dashboard:
ERROR:  https://dashboard.ngrok.com/agents
ERROR:  
ERROR:  ERR_NGROK_108
ERROR:  https://ngrok.com/docs/errors/err_ngrok_108
ERROR:  
